In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime

# Step 1 – Read config
config_df = spark.read.option("header", "true").csv("/Volumes/practice/default/practice/employee.csv").collect()[0]
file_path = config_df["loca"]
stage_table = config_df["tablename1"]
raw_table = config_df["tablename2"]

# Step 2 – Read source data
src_df = spark.read.option("header", "true").csv(file_path)

# Step 3 – Load into Stage (truncate)
src_df.write.mode("overwrite").saveAsTable(stage_table)

# Step 4 – Prepare stage for merge (add update date)
current_date = datetime.now().strftime("%d/%m")
stage_df = src_df.withColumn("last_updated_date", F.lit(current_date))

# Step 5 – Merge into Raw
if spark._jsparkSession.catalog().tableExists(raw_table):
    raw_delta = DeltaTable.forName(spark, raw_table)
    raw_delta.alias("raw").merge(
        stage_df.alias("stg"),
        "raw.name = stg.name"
    ).whenMatchedUpdate(
        set={"last_updated_date": F.lit(current_date)}
    ).whenNotMatchedInsertAll().execute()
else:
    stage_df.write.mode("overwrite").saveAsTable(raw_table)

print("ETL process completed successfully.")
